<a href="https://colab.research.google.com/github/osmanatam/Advanced-Deep-Learning-with-Keras/blob/master/ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install spacy

In [0]:
import random
import json
import spacy
from spacy import displacy

## Veri seti
Kullandığımız veriseti 77 farklı kategoride varlık ismi içeren TWNERTC veri setinin astronomi konulu kısmı. Veri setinin tamamına ve araştırmacıların orjinal makalesine [buradan](https://data.mendeley.com/datasets/cdcztymf4k/1) ulaşabilirsiniz.

Veri setinin temizlenmiş ve json formatındaki halini ise [buradan](https://drive.google.com/file/d/1o0j4UcEBCehwJSG2SHOl_I-h8TTA6pdI/view?usp=sharing) indirebilirsiniz.


Kullanacağımız astronomi temalı veri setinde 12589 cümle ve etiketlenmiş varlık isimleri mevcut.


In [0]:
from google.colab import files
uploaded = files.upload()

Saving data_astronomy.json to data_astronomy.json


In [0]:
with open('data_astronomy.json', 'r') as fp:
    data = json.load(fp)

# Astronomi konulu örnek girdi
print(dict(list(data['astronomy'].items())[150:151]))

{"Betelgeuse'un üstündeki yıldızlar avcının sağ kolunu Bellatrix'den ötede olan yıldızlarda avcının kalkanını oluşturur .": {'entities': [[0, 12, 'B-star_name'], [24, 32, 'B-celestial_object_category'], [78, 88, 'B-celestial_object_category']]}}


## SpaCy NER
[SpaCy](https://spacy.io/api) çok amaçlı ve Türkçe'yi kısmen destekleyen bir doğal dil işleme kütüphanesidir. 

- Eğitilmiş dil modelleri
- Dil işleme araçları yanı sıra
- Bağlılık analizi (dependency parsing), varlık ismi tanıma (NER) gibi görevler için model eğitim arayüzü


SpaCy Varlık İsmi Tanıma (NER) modeli kullandığı yaklaşım:
- https://www.youtube.com/watch?time_continue=3&v=sqDHBH9IjRU&feature=emb_logo

![SpaCy NER formatı](https://miro.medium.com/max/1216/1*TZWuuZjHDMWArQUCohXWGg.png)

In [0]:
def convert_to_spacy_format(data):
    DATA = []
    for k in data['astronomy']:
    	ents = data['astronomy'][k]['entities']
    	for i in range(len(ents)):
    		DATA.append((k, {'entities': [(
                ents[i][0], int(ents[i][1])+1, ents[i][2])]}))
    return DATA

In [0]:
def train_spacy(data, iterations):
    TRAIN_DATA = data
    # Boş Language objesi yarat
    nlp = spacy.blank('tr')

    # NER pipeline'ı oluştur
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
       
    # Varlık ismi etiketlerini ekle
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # NER haricindeki pipeline'ları dondur
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  
        optimizer = nlp.begin_training()

        # Modeli eğit
        for itn in range(iterations):
            print("Iteration: {}".format(itn))
            random.shuffle(data)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  
                    [annotations], 
                    drop=0.2, 
                    sgd=optimizer, 
                    losses=losses)
            print(losses)
    return nlp

In [0]:
# Modeli eğit ve kaydet
data = convert_to_spacy_format(data)
model = train_spacy(data[:400], 10)
model.to_disk("astro-ner")

Iteration: 0
{'ner': 933.7851642363321}
Iteration: 1
{'ner': 774.6200844128067}
Iteration: 2
{'ner': 762.9437644514078}
Iteration: 3
{'ner': 749.4842491692067}
Iteration: 4
{'ner': 750.1285597287408}
Iteration: 5
{'ner': 758.3218601304787}
Iteration: 6
{'ner': 755.4335308796225}
Iteration: 7
{'ner': 730.3519379900898}
Iteration: 8
{'ner': 741.1564092741851}
Iteration: 9
{'ner': 747.9061720179953}


In [0]:
# Kaydedilmiş modeli yükle ve test et
nlp = spacy.load('astro-ner')

test_text = "Betelgeuse'un üstündeki yıldızlar avcının sağ kolunu Bellatrix'den ötede olan yıldızlarda avcının kalkanını oluşturur ."
doc = nlp(test_text)

# Tespit edilen varlık isimleri
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Betelgeuse'un 0 13 B-star_name
yıldızlarda 78 89 B-celestial_object_category


In [0]:
# Displacy modülü ile görselleştirme
displacy.render(doc, style="ent", jupyter=True)